<a href="https://colab.research.google.com/github/john-d-noble/callcenter/blob/main/2%20CB_Step_7_Deep_Learning_and_Hybrid_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy scikit-learn tensorflow prophet xgboost neuralprophet

In [ ]:
# Inspect column names in the CSV file
df_inspect = pd.read_csv('all_combined_data.csv')
print(df_inspect.columns)

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from prophet import Prophet
from xgboost import XGBRegressor
from neuralprophet import NeuralProphet
import torch  # Added for safe_globals
# Fix for UnpicklingError: Import the actual class and add to safe_globals
from neuralprophet.configure import ConfigSeasonality
torch.serialization.add_safe_globals([ConfigSeasonality])

# Step 1: Load the data from CSV file with explicit date format to suppress parsing warning
df = pd.read_csv('all_combined_data.csv', parse_dates=['Unnamed: 0'], date_format='%m/%d/%y', index_col='Unnamed: 0')

# Assume 'Calls' is the target column
target = 'Calls'

# Prepare data: Sort by date if not already
df = df.sort_index()

# Feature Engineering (similar to ML: lags, rollings, dummies)
df['Lag1'] = df[target].shift(1)
df['Lag7'] = df[target].shift(7)
df['Rolling_Mean_7'] = df[target].rolling(window=7).mean()
df['Rolling_Std_7'] = df[target].rolling(window=7).std()
df = pd.get_dummies(df, columns=['DayOfWeek'], drop_first=True)

# Drop NaNs
df = df.dropna()

# Get the list of feature columns AFTER feature engineering and dummy variable creation
feature_columns = df.drop(columns=[target]).columns.tolist()

# Time series cross-validation: 5 splits
tscv = TimeSeriesSplit(n_splits=5)

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100  # As percentage
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

# Dictionary to store average metrics for each model
model_metrics = {}

# Helper to create sequences for LSTM (timesteps=7)
def create_sequences(data, timesteps=7):
    X_seq, y_seq = [], []
    # Features are all columns except the target
    feature_cols_lstm = [col for col in data.columns if col != target]
    for i in range(len(data) - timesteps):
        X_seq.append(data[feature_cols_lstm].iloc[i:i+timesteps].values)
        y_seq.append(data[target].iloc[i+timesteps])  # Predict next value
    return np.array(X_seq), np.array(y_seq)

# 1. LSTM Network
lstm_preds_inv = []
lstm_trues_inv = []
for train_idx, test_idx in tscv.split(df):
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    # Scalers: Fit on train only
    feature_scaler = MinMaxScaler()
    feature_scaler.fit(train_df[feature_columns])

    target_scaler = MinMaxScaler()
    target_scaler.fit(train_df[[target]])  # Fit as 2D array

    # Scale train and test
    train_scaled = pd.DataFrame(feature_scaler.transform(train_df[feature_columns]),
                                index=train_df.index, columns=feature_columns)
    train_scaled[target] = target_scaler.transform(train_df[[target]])

    test_scaled = pd.DataFrame(feature_scaler.transform(test_df[feature_columns]),
                               index=test_df.index, columns=feature_columns)
    test_scaled[target] = target_scaler.transform(test_df[[target]])

    # Create sequences
    X_train_seq, y_train_seq = create_sequences(train_scaled)
    X_test_seq, y_test_seq = create_sequences(test_scaled)

    # Build LSTM model with Input layer to suppress warning
    model = Sequential()
    model.add(Input(shape=(7, X_train_seq.shape[2])))  # Use shape of X_train_seq
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Fit
    model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, verbose=0)

    # Predict (scaled)
    pred_scaled = model.predict(X_test_seq, verbose=0)

    # Inverse scale predictions and trues
    pred_inv = target_scaler.inverse_transform(pred_scaled)
    true_inv = target_scaler.inverse_transform(y_test_seq.reshape(-1, 1))

    lstm_preds_inv.extend(pred_inv.flatten())
    lstm_trues_inv.extend(true_inv.flatten())

lstm_metrics = calculate_metrics(lstm_trues_inv, lstm_preds_inv)
model_metrics['LSTM'] = lstm_metrics

# 2. Neural Prophet (with matplotlib backend to avoid Plotly error)
np_preds = []
np_trues = []
for train_idx, test_idx in tscv.split(df):
    train_temp = df.iloc[train_idx].reset_index()
    date_col = train_temp.columns[0]  # The added index column (date)
    train_df = train_temp.rename(columns={date_col: 'ds', 'Calls': 'y'})

    test_temp = df.iloc[test_idx].reset_index()
    test_df = test_temp.rename(columns={date_col: 'ds', 'Calls': 'y'})  # Use same date_col name assuming consistent

    # Fit NeuralProphet with matplotlib and explicit seasonalities
    model = NeuralProphet(epochs=50, batch_size=32, learning_rate=0.01,  # Manual LR to avoid finder warning
                          yearly_seasonality=False, daily_seasonality=False)  # Explicit to suppress auto warnings
    model.set_plotting_backend('matplotlib')
    model.fit(train_df[['ds', 'y']], freq='D')

    # Make future dataframe
    future = model.make_future_dataframe(train_df[['ds', 'y']], periods=len(test_df))

    # Predict
    forecast = model.predict(future)
    pred = forecast['yhat1'].tail(len(test_df)).values
    np_preds.extend(pred)
    np_trues.extend(test_df['y'])

np_metrics = calculate_metrics(np_trues, np_preds)
model_metrics['Neural Prophet'] = np_metrics

# 3. Hybrid: Prophet + XGBoost (Prophet for trend/seasonal, XGBoost on residuals)
hybrid_preds = []
hybrid_trues = []
for train_idx, test_idx in tscv.split(df):
    train_df_hybrid = df.iloc[train_idx]
    test_df_hybrid = df.iloc[test_idx]

    # Step 1: Fit Prophet
    # Prepare data for Prophet
    prophet_train_temp = train_df_hybrid.reset_index()
    date_col = prophet_train_temp.columns[0]  # The added index column (date)
    prophet_train_df = prophet_train_temp.rename(columns={date_col: 'ds', 'Calls': 'y'})

    prophet_model = Prophet(weekly_seasonality=True)
    prophet_model.fit(prophet_train_df[['ds', 'y']])

    # Predict on train for residuals
    train_future_prophet = prophet_model.make_future_dataframe(periods=0)
    train_forecast_prophet = prophet_model.predict(train_future_prophet)
    train_residuals = train_df_hybrid['Calls'] - train_forecast_prophet['yhat'].values

    # Predict on test for base forecast
    test_future_prophet = prophet_model.make_future_dataframe(periods=len(test_df_hybrid))
    test_forecast_prophet = prophet_model.predict(test_future_prophet)
    prophet_test_pred = test_forecast_prophet['yhat'].tail(len(test_df_hybrid)).values

    # Step 2: Fit XGBoost on residuals using features
    # Select features for XGBoost after feature engineering
    xgb_features = [col for col in train_df_hybrid.columns if col != target]
    X_train = train_df_hybrid[xgb_features]
    y_res_train = train_residuals
    xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    xgb_model.fit(X_train, y_res_train)

    # Predict residuals on test
    X_test = test_df_hybrid[xgb_features]
    res_pred = xgb_model.predict(X_test)

    # Combine: Prophet pred + residual pred
    pred = prophet_test_pred + res_pred
    hybrid_preds.extend(pred)
    hybrid_trues.extend(test_df_hybrid[target])

hybrid_metrics = calculate_metrics(hybrid_trues, hybrid_preds)
model_metrics['Prophet + XGBoost Hybrid'] = hybrid_metrics

# Summarize performance
print("\nModel Performance Summary:")
metrics_df = pd.DataFrame(model_metrics).T
print(metrics_df)

# Pick winner: Lowest MAE (primary metric)
winner = metrics_df['MAE'].idxmin()
print(f"\nChampion DL/Hybrid Model: {winner}")
print(f"Metrics: {metrics_df.loc[winner].to_dict()}")

WARNING - (NP.plotting.log_warning_deprecation_plotly) - DeprecationWarning: default plotting_backend will be changed to plotly in a future version. Switch to plotly by calling `m.set_plotting_backend('plotly')`.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.65% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.65% of the data.
WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/neuralprophet/df_utils.py:1152: 

Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.65% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.65% of the data.
WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/p

Predicting: 9it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
WARNING - (NP.plotting.log_warning_deprecation_plotly) - DeprecationWarning: default plotting_backend will be changed to plotly in a future version. Switch to plotly by calling `m.set_plotting_backend('plotly')`.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.824% of the data.
INFO:NP.df_utils:Major frequency D corresponds to

...

Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.929% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.929% of the data.
WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib

Predicting: 45it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmphsnroqlb/cc2py25l.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphsnroqlb/v6vx1jaq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=99588', 'data', 'file=/tmp/tmphsnroqlb/cc2py25l.json', 'init=/tmp/tmphsnroqlb/v6vx1jaq.json', 'output', 'file=/tmp/tmphsnroqlb/prophet_modelrc9jon0f/prophet_model-20250909024623.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
02:46:23 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
02:46:


Model Performance Summary:
                                  MAE         RMSE       MAPE
LSTM                      1633.927683  2351.441419  23.327835
Neural Prophet            1725.359604  2271.918380  22.287731
Prophet + XGBoost Hybrid  2051.420178  2865.172197  28.284329

Champion DL/Hybrid Model: LSTM
Metrics: {'MAE': 1633.9276827162591, 'RMSE': 2351.441419486085, 'MAPE': 23.32783530216595}
